In [19]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
from glob import glob
import os

### Get file paths

In [2]:
folder_path = r"C:\Users\lukas\Desktop\spiral_basic_in_multipillar175x50_002"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]

mesh = get_mesh(msh_file)
mesh = ground_mesh(mesh)
magnetisation, _ = get_magnumfe_magnetisation(mag_files[0])

### Make sure that the projection vector is correct and that the structure is oriented well

In [26]:
struct = get_struct_from_mesh(mesh)
points, faces, tetra = get_mesh_data(mesh)
struct.faces = frozen_to_numpy(get_edge_faces(tetra))

In [42]:
# need to mirror the structure
struct.vertices[:, 0] *= -1
projected_structure = project_structure(struct, phi=90)

In [44]:
vis = MagnumfeVisualizer(struct, projected_structure)
vis.show()

### Get the projection and piercing information and save immediately. Load if exists

In [45]:
piercings_file = os.path.join(data_folder, "piercings.npy")
if os.path.exists(piercings_file):
    struct, struct_projected, piercings_list, p, mesh = load_piercing_data(piercings_file)
else:
    struct_projected, piercings_list, p = get_projection_with_piercings(mesh)
    save_piercing_data(piercings_file, struct, struct_projected, piercings_list, p, mesh)

MemoryError: Unable to allocate 105. MiB for an array with shape (3, 1532568, 3) and data type float64